# Image Directory
We've provided a few images from ImageNet. If working locally, update ```DATA_FOLDER``` based on your local directory. Based on the documentation for [ImageFolder](https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html), within ```DATA_FOLDER``` you should have one folder for each class, respectively ```'bagel', 'barn', 'goldfish', 'mud_turtle'```. Put each provided image into the respective folder.

We've also provided a file that lists the class names of ImageNet, in order. Update ```CLASSNAMES_FILE``` based on your local directory.

In [1]:
DATA_FOLDER = './imagenet-images/'
CLASSNAMES_FILE = './imagenet_classnames.txt'

If you're using Colab, we've noticed an issue with hidden files, so uncomment and run the following line:

In [ ]:
# rm -rf /content/imagenet-images/.ipynb_checkpoints

# Imports

If you're running this notebook in Colab, you'll want to uncomment and run the following line.

If you're running this notebook locally or on a Grace cluster, you can separately install any packages you use. Note: if your device is GPU-compatible, you'll likely get a significant speed-up in running code for this assignment and the next, but it shouldn't be *strictly* necessary.

In [ ]:
# !pip install captum

In [2]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms.v2 as transforms

from torchvision import models

from captum.attr import IntegratedGradients
from captum.attr import visualization as viz

ModuleNotFoundError: No module named 'captum'

# Useful methods
Just run these cells, they contain functions that may be useful for visualizing output in particular.

```means``` and ```stds``` are defined in a later cell.

Notice that ```visualize_attributions``` accepts a List[] of **numpy arrays**.

In [ ]:
def unnormalize(img):
    r"""
    Args:
        img (Tensor): Tensor image of size (C, H, W), normalized according to means, stds.
    Returns:
        numpy.ndarray: Unnormalized image of size (H, W, C) in range [0, 1].
    """
    # Convert the Tensor img to a numpy array that can be visualized
    img = img.cpu().permute(1, 2, 0).numpy()
    img = (img * stds[None,None]) + means[None,None]
    img = np.clip(img, a_min=0.0, a_max=1.0)
    return img

In [ ]:
def visualize_attributions(predicted_class, predicted_probability, image, attrs, methods, titles):
    r"""
    Output n results, each for the same input image.
    attrs, methods, titles should be the same length, n.
    Args:
        predicted_class (str): Name of class predicted
        predicted_probability (float): Probability assigned to predicted class by model
        image (Tensor): Original image to unnormalize.
        attrs (List[]), methods(List[]), titles (List[]):
            See https://captum.ai/api/utilities.html for documentation on
            captum.attr.visualization.visualize_image_attr.
            Elements of attrs should be numpy arrays of shape (224, 224, 3)
            Use 'blended_heat_map' or 'original_image' for methods.
    """
    print(f"Predicted: {predicted_class} with probability: {predicted_probability}")

    original_image = unnormalize(image.cpu().detach())

    for attr, method, title in zip(attrs, methods, titles):
        if method == 'original_image':
            _ = viz.visualize_image_attr(
                attr,
                original_image,
                method=method,
                title=title,
            )
        else:
            _ = viz.visualize_image_attr(
                attr,
                original_image,
                method=method,
                sign="all",
                show_colorbar=True,
                title=title,
            )

# Implementation
Here's where you start coding! You will implement the IntegratedGradients [IG](https://arxiv.org/abs/1703.01365) algorithm in the method ```ig_attribution```. The purpose of this exercise is teach you the IG algorithm, as well as to get you accustomed to accessing gradients in PyTorch, which might be helpful on your projects. Feel free to design any helper functions you might need!

I'll bold arguments to the method in this introduction to help you map the description to what you will implement:

IG explains why a **model** predicts a **target class** for a given **input image**. It defines a path from a **baseline image** (for us, the all-zero image) to the input image, and integrates the gradient of the model's score with respect to the path. We will use two different methods of computing [Riemann sums](https://en.wikipedia.org/wiki/Riemann_sum) to approximate the integral of this gradient along a linear path.

To illustrate the difference between the methods, consider approximating the integral of $f(x)=x^3$ from 0 to 25 with 5 subintervals.\
```method_name='riemann',method_side='left'``` will use "rectangles" of
$$(height,start,end)=(0^3,0,5),(5^3,5,10),(10^3,10,15),\dots,(20^3,20,25).$$
```method_name='riemann', method_side='right'``` will use "rectangles" of
$$(height,start,end=(5^3,0,5),(10^3,5,10),(15^3,10,15),\dots,(25^3,20,25).$$
```method_name='quad_linear',method_side='left'``` will use "rectangles" of
$$(height,start,end)=(0^3,0,1),(1^3,1,4),(4^3,4,9),\dots,(16^3,16,25).$$
```method_name='quad_linear',method_side='right'``` will use "rectangles" of
$$(height,start,end)=(1^3,0,1),(4^3,1,4),(9^3,4,9),\dots,(25^3,16,25).$$
That is, ```method_name='quad_linear'``` will simply change the rate at which we move along the **still-linear** path; equivalently, it will change the widths of the subintervals to be non-constant.

By independence of path, the sum of the values of the integrated gradients $sum(IG)$ should equal $model(input)-model(baseline).$ The output ```delta``` is the difference from approximating, so it should be
$$delta=sum(IG) - (model(input)-model(baseline)).$$


In [ ]:
def ig_attribution(model, inp, baseline, target_class, method_name, method_side, n_steps):
    r"""
    Args:
        model (nn.Module): The trained model to be explained.
        inp (Tensor): The input to the model.
        baseline (Tensor): The baseline input to compare with.
        target_class (int): Output index for which gradients are computed.
        method_name (str): Method for approximating the integral, one of `riemann` or `quad_linear`.
        method_side (str): Method for approximating the integral, one of `left` or `right`.
        n_steps (int): The number of steps to use in the Riemann approximation.
    Returns:
        attributions (Tensor): Integrated gradients with respect to each input feature. Same shape as inp.
        delta (float): The difference between the total approximated and true integrated gradients.
    """

    ### YOUR CODE HERE:

    return attribution, delta

# Data and Model
## Data Processing
We've included a few images from the ImageNet dataset.


In [ ]:
# Transformations applied to images before passing them to the model
# Pretrained normalization based on https://discuss.pytorch.org/t/how-to-preprocess-input-for-pre-trained-networks/683
means, stds = [0.485, 0.456, 0.406], [0.485, 0.456, 0.406]
means, stds = np.array(means), np.array(stds)

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToImage(), # Converts to tensor
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Normalize(mean=means, std=stds)
    ]
)

In [ ]:
# Load the small dataset to interpret
dataset = torchvision.datasets.ImageFolder(DATA_FOLDER, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
# The class names in the dataset are not quite aligned with the class names in the model
my_classes = ('bagel', 'barn', 'goldfish', 'mud_turtle')
classes = []
with open(CLASSNAMES_FILE, 'r') as f:
    for line in f:
        classes.append(line.strip())

## Load model
Import torchvision's ResNet18 model. Be sure to use the pretrained weights!

In [ ]:
### YOUR CODE HERE:
### Set the model to eval mode so we can interpret it
net = ...

## Quick Check
We'll visualize the images, check their ground truths, and output the pretrained ResNet model's predictions (highest-scoring classes) for them.

In [ ]:
# Load images and labels from the dataset
images_ = torch.stack([dataset[i][0] for i in range(len(my_classes))])
labels_ = [dataset[i][1] for i in range(len(my_classes))]

# Show images
plt.imshow(unnormalize(torchvision.utils.make_grid(images_)))
plt.show()
print("GroundTruth: ", " ".join("%5s" % my_classes[labels_[j]] for j in range(len(my_classes))))

# Predictions

outputs_ = net(images_)
_, predicted_ = torch.max(outputs_, 1)
print("Predicted: ", " ".join("%5s" % classes[predicted_[j]] for j in range(len(my_classes))))

# Testing

## Config


In [ ]:
# Current method to be used for captum visualization
# See https://captum.ai/api/integrated_gradients.html for documentation,
# especially the attribute method.
cur_method_captum = 'riemann_left'
# Current method to be used for self-implemented visualization
cur_method_name, cur_method_side = 'riemann', 'left'

# Choose an image to test on
ind_ = 3
input_tns_ = images_[ind_].unsqueeze(0).requires_grad_()

## Using the Captum library's IG implementation
Use the [documentation](https://captum.ai/api/integrated_gradients.html) for Captum's IG implementation and complete the following function.

In [ ]:
def captum_ig(model, inp, baseline, target_class, method, n_steps=50):
    r"""
    Args:
        model (nn.Module): The trained model to be explained.
        inp (Tensor): The input to be explained
        baseline (Tensor): The baseline input to compare with.
        target_class (int): Output index for which gradients are computed.
        method (str): Method for approximating the integral, see Captum documentation for details.
        n_steps (int): The number of steps to use in the integral approximation.
    Returns:
        attributions (Tensor): Integrated gradients with respect to each input feature. Same shape as inp.
        delta (float): The difference between the total approximated and true integrated gradients.
    """

    ### YOUR CODE HERE:

    return attr_ig, delta

Once you've done this, the following code block should run:

In [ ]:
attr_ig_captum_, delta_captum_ = captum_ig(net, input_tns_, input_tns_ * 0, labels_[ind_], cur_method_captum, 5)
# what's the lower bound on error of the approximated integral
print("Approximation delta: ", abs(delta_captum_))

## Using your implementation of IG
If you've done ```ig_attribution``` right, ```self_ig``` should essentially look like the ```captum_ig```.

In [ ]:
def self_ig(model, inp, baseline, target_class, method_name, method_side, n_steps=50):
    r"""
    Args:
        model (nn.Module): The trained model to be explained.
        inp (Tensor): The input to the model.
        baseline (Tensor): The baseline input to compare with.
        target_class (int): Output index for which gradients are computed.
        method_name (str): Method for approximating the integral, one of `riemann` or `quad_linear`.
        method_side (str): Method for approximating the integral, one of `left` or `right`.
        n_steps (int): The number of steps to use in the Riemann approximation.
    Returns:
        attributions (Tensor): Integrated gradients with respect to each input feature. Same shape as inp.
        delta (float): The difference between the total approximated and true integrated gradients.
    """

    ### YOUR CODE HERE:

    return attr_ig, delta

Once you've done this, the following code block should run, and the output should be the same as the approximation delta from ```captum_ig```.

In [ ]:
attr_ig_, delta_ = self_ig(net, input_tns_, input_tns_ * 0, labels_[ind_], cur_method_name, cur_method_side, 5)
print("Approximation delta: ", abs(delta_))

# Comparison
Choose one of the given 4 images and choose an image of your own.

Use Captum (a ```riemann``` method), your implementation of ```riemann```, and your implementation of ```quad_linear``` to explain each image. Be sure to visualize the image and each explanation (```visualize_attributions``` should help) and check the approximation deltas. Comment on the results. It may be interesting to try with the ground truth, the predicted class, or a random class as the target.

(You can choose whether to use right or left -- use the same side for a given image, though.)

(We used 50 steps. You may use however many steps you'd like -- be consistent.)

In [ ]:
### YOUR CODE HERE: